## Informações Gerais
 | Informações | Detalhes |
 |------------|-------------|
 |Nome Tabela | gold.IndicadorPaisProduto |
 |Origem | silver.vendas |

## Histórico de Atualizações
 | Data | Desenvolvido por | Motivo |
 |:----:|--------------|--------|
 |16/07/2024 | Patrick Diorio  | Criação do notebook |
 |20/08/2024 | Patrick Diorio  | inclusão coluna valor |

In [0]:
%run /Workspace/Repos/AGRICOPEL/Agricopel-Databricks/FINANCEIRO/CONTAS_RECEBER/00.global/functions

In [0]:
from pyspark.sql.functions import current_date, current_timestamp, expr

In [0]:
database = "gold.financeiro"
tabela = "IndicadorTipoCliente"

In [0]:
dfPaisProduto = spark.sql("""
    select
        nm_pais,
        nm_produto,
        sum(nr_valor_total) as nr_valor_total,
        current_timestamp() as dt_data_extracao
    from silver.sistema_vendas.vendas
    group by all
""")

In [0]:
dfPaisProduto = dfPaisProduto.withColumn("data_carga", current_date())
dfPaisProduto = dfPaisProduto.withColumn("data_hora_carga", expr("current_timestamp() - INTERVAL 3 HOURS"))

In [0]:
comentario_tabela = 'Este é um indicador para a área financeira contendo a quantidade e total de vendas por tipo de cliente'

lista_comentario_colunas = {
'nm_pais' :  'Nome do país.',
'nm_produto' : 'Nome do produto.',
'nr_valor_total' : 'Valo total da venda.',
'dt_data_extracao' : 'data extração bronze.',
'data_carga' : 'Data que o registro foi carregado',
'data_hora_carga' : 'Data e hora que o registro foi carregado'
}

In [0]:
dfPaisProduto.write \
          .format('delta') \
          .mode('overwrite') \
          .option('overwriteSchema', 'true') \
          .saveAsTable(f'{database}.{tabela}')
adicionaComentariosTabela(database, tabela, comentario_tabela, lista_comentario_colunas)
print("Dados gravados com sucesso!")

In [0]:
spark.sql(f"OPTIMIZE {database}.{tabela}")
print(f"Processo de otimização finalizado!.")